In [ ]:
!pip install plotly

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [3]:
max_vout = 1.0  # Vout max

amp_r1 = 82e3

df = pd.DataFrame(range(1, 7), columns=['Npanel'])
df['Vpanel [V]'] = df['Npanel'] * 23
df['r2 [Ohm]'] = amp_r1 / (df['Vpanel [V]'] / max_vout - 1)
df['Ipanel [A]'] = df['Vpanel [V]'] / (amp_r1 + df['r2 [Ohm]'])
df['Ppanel [W]'] = df['Vpanel [V]'] * df['Ipanel [A]']
df['ADC_res [V/LBS]'] = (max_vout / 1024) * df['Vpanel [V]'] / max_vout
df

,Npanel,Vpanel [V],r2 [Ohm],Ipanel [A],Ppanel [W],ADC_res [V/LBS]
0,1,23,3727.272727,0.000268,0.006171,0.022461
1,2,46,1822.222222,0.000549,0.025244,0.044922
2,3,69,1205.882353,0.000829,0.057220,0.067383
3,4,92,901.098901,0.001110,0.102098,0.089844
4,5,115,719.298246,0.001390,0.159878,0.112305
5,6,138,598.540146,0.001671,0.230561,0.134766


In [47]:
max_vout = 1

amp_r2 = 1.5e3
div_r1 = 100e3
div_r2 = 680

df = pd.DataFrame(range(1, 8), columns=['Npanel'])
df['Vpanel [V]'] = df['Npanel'] * 23
df['Iloss [mA]'] = 1000 * df['Vpanel [V]'] / (div_r1 + div_r2)
df['Ploss [W]'] = (df['Vpanel [V]'] ** 2 ) / (div_r1 + div_r2)
df['Vdiv [V]'] = df['Vpanel [V]'] * div_r2 / (div_r1 + div_r2)
df['Avf_target'] = round(max_vout / df['Vdiv [V]'], 3)

df['amp_r1 [ohm]'] = (df['Avf_target'] - 1) * amp_r2
df

,Npanel,Vpanel [V],Iloss [mA],Ploss [W],Vdiv [V],Avf_target,amp_r1 [ohm]
0,1,23,0.228447,0.005254,0.155344,6.437,8155.5
1,2,46,0.456893,0.021017,0.310687,3.219,3328.5
2,3,69,0.685340,0.047288,0.466031,2.146,1719.0
3,4,92,0.913786,0.084068,0.621375,1.609,913.5
4,5,115,1.142233,0.131357,0.776718,1.287,430.5
5,6,138,1.370679,0.189154,0.932062,1.073,109.5
6,7,161,1.599126,0.257459,1.087406,0.920,-120.0


In [35]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])
fig = go.Figure()

# Add trace for 'Ploss [W]' on the primary y-axis
fig.add_trace(
    go.Scatter(x=df['Vpanel [V]'], y=df['Ploss [W]'], name='Potencia disipada', line=dict(color='blue')),
    # secondary_y=False,
)

# Add trace for 'Avf_target' on the secondary y-axis
fig.add_trace(
    go.Scatter(x=df['Vpanel [V]'], y=df['Avf_target'], 
        name='Ganancia necesaria',
        line=dict(color='green'),
        yaxis='y2'
    ),
    # secondary_y=True,
)

# Add trace for 'amp_r1 [ohm]' on the secondary y-axis
fig.add_trace(
    go.Scatter(x=df['Vpanel [V]'], y=df['amp_r1 [ohm]'],
        name='Resistencia pot digital',
        line=dict(color='red'),
        yaxis='y3'
    ),
    # secondary_y=True,
)


fig.update_layout(
    title_text="Resumen valores monitor solar",
    yaxis=dict(title="P_loss [W]", titlefont=dict(color="blue"),
        range=[0, df['Ploss [W]'].max() * 1.05],
    ),
    yaxis2=dict(
        title="Ganancia No inversor",
        titlefont=dict(color="green"),
        overlaying="y",
        range=[0, df['Avf_target'].max() * 1.05],
        side="right"
        ),
    yaxis3=dict(
        title="Potenciometro digital [ohm]",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        anchor="free",
        overlaying="y",
        range=[0, df['amp_r1 [ohm]'].max() * 1.05],
        # position=1,
        autoshift=True
    ),
    xaxis=dict(title="Vpanel [V]")
)


fig.show()

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Vpanel [V]'], y=df['Ploss [W]'], name="yaxis data"))

fig.add_trace(go.Scatter(x=df['Vpanel [V]'], y=df['Avf_target'], name="yaxis2 data", yaxis="y2"))

fig.add_trace(
    go.Scatter(x=df['Vpanel [V]'], y=df['amp_r1 [ohm]'], name="yaxis3 data", yaxis="y3")
)

# fig.add_trace(
#     go.Scatter(x=[3, 4, 5], y=[400, 500, 600], name="yaxis4 data", yaxis="y4")
# )


fig.update_layout(
    # xaxis=dict(domain=[0.25, 0.75]),
    yaxis=dict(
        title="yaxis title",
    ),
    yaxis2=dict(
        title="yaxis2 title",
        overlaying="y",
        side="right",
    ),
    yaxis3=dict(title="yaxis3 title", anchor="free", overlaying="y", autoshift=True),
    # yaxis4=dict(
    #     title="yaxis4 title",
    #     anchor="free",
    #     overlaying="y",
    #     autoshift=True,
    # ),
)

fig.update_layout(
    title_text="Shifting y-axes with autoshift",
)

fig.show()